In [1]:
from extras.fileOperations import getFileNamesInDir
from compmusic.extractors.makam import metadata as md
import os
import sys
import codecs

reload(md)

<module 'compmusic.extractors.makam.metadata' from '/home/sertansenturk/Documents/notaIcra/code/pycompmusic/compmusic/extractors/makam/metadata.pyc'>

In [2]:
txtfolder = '../../data/SymbTr/'
[txtfiles, baseTemp, symbtrnames] = getFileNamesInDir(txtfolder, '.txt')
symbtrnames = [os.path.splitext(s)[0] for s in symbtrnames]

metadataFolder = './metadata/'
metadataFiles = [os.path.join(metadataFolder, s) for s in symbtrnames]

In [12]:
## get section metadata 

# stdout is directed to file to be manipulated later
old_stdout = sys.stdout
sys.stdout = codecs.open('log.txt', 'w', "utf-8")

# loop through all scores
for f, mf, s in zip(txtfiles, metadataFiles, symbtrnames):
    print s
    metadata = md.extract(f, s)
    with open(mf, 'w') as text_file:
        text_file.write(metadata)
        
# return to old stdout
sys.stdout = old_stdout

print "Extraction complete!"

Extraction complete!


In [14]:
prevLineWarn = False
prevNoLyricsWarn = False
warn_log = []
no_lyrics_log = []
for line in reversed(open("log.txt").readlines()):
    if line == '    Lyrics is empty. Cannot determine the sections\n': # no lyrics, no need to copy
        prevNoLyricsWarn = True
        prevLineWarn = False
    elif prevNoLyricsWarn: # score with no lyrics
        prevNoLyricsWarn = False
        prevLineWarn = False
        no_lyrics_log.append(line)
    elif line[0] == ' ': # warning, copy
        prevNoLyricsWarn = False
        prevLineWarn = True
        warn_log.append(line)
    elif prevLineWarn: # score with warning
        prevNoLyricsWarn = False
        prevLineWarn = False
        warn_log.append(line)
    else: # score wo warning
        prevNoLyricsWarn = False
        prevLineWarn = False

with open('warn_log.txt', 'w') as ll:
    ll.writelines(reversed(warn_log))
    
with open('no_lyrics_log.txt', 'w') as ll:
    ll.writelines(reversed(no_lyrics_log))
        

'huseyni--sarki--agiraksak--bendeni_gordukce--ismet_aga\n'

In [16]:
import Levenshtein

Levenshtein.distance('huseyni--sarki--agiraksak--bendeni_gordukce--ismet_aga', 'huseyni--sarkiasdfagiraksakadsf--bendeni_gordukce--ismet_aga')

8